In [1]:
import sys

print(sys.version)

3.11.3 | packaged by conda-forge | (main, Apr  6 2023, 08:58:31) [Clang 14.0.6 ]


## 1. 디스크립터 복습

## 디스크립터 클래스와 활용 클래스 정의

In [2]:
class descriptor:
    
    def __init__(self, value):
        self.value = value
    # self : 디스크립터 객체
    # instance : 디스크립터를 사용하는 클래스의 객체
    def __get__(self, instance, owner):
        return self.value
    
    def __set__(self, instance, value):
        self.value = value
        
    def __delete__(self, instance):
        del self.value

In [3]:
class A:
    #data = 10
    data = descriptor(10)

### 디스크립터 정의된 속성 내의 메소드 참조

In [4]:
a = A()
a.data

10

### 실제 데이터를 검색하는 방식

In [5]:
# == a.data
type(a).__dict__['data'].__get__(a, type(a))

10

## 실습

### 숫자를 저장하는 디스크립터 만들기

1. `__get__`, `__set__`, `__delete__` 구현
1. `__set__`을 통해 숫자를 저장한다.
1. 숫자는 호출할때마다 +1 증가
1. 숫자를 할당할때마다, 이전값에 더 합니다.(덮어쓰기 x)
1. 숫자를 지울떄는 최근에 추가한 수만큼만 제거합니다. (추가한 숫자들의 목록을 별도 저장)
1. 다음과 같이 작동해야합니다.


### 디스크립터 클래스를 사용해서 특정 값 처리



In [6]:
class descriptor:
    # instance -> class A 인스턴스
    # self -> 디스크립터 클래스 인스턴스
    def __get__(self, instance, owner):
        instance.value += 1
        return instance.value
    
    def __set__(self, instance, value):
        if not hasattr(self, 'value'):
            instance.value = 0
            instance.recent = []
        instance.value += value
        instance.recent.append(value)
        
    def __delete__(self, instance):
        instance.value -= instance.recent[-1]
        instance.recent.pop()
        # 최근 추가된 값만큼만 마이너스(-)
        # 그리고 최근 추가된 리스트에서 제거
    
    

In [7]:
class A:
    number = descriptor()

In [8]:
a = A()
b = A()

In [9]:
a.__dict__

{}

In [10]:
b.__dict__

{}

### 인스턴스에 속성 추가

In [11]:
a.number = 1
print(a.number)

2


In [12]:
a.__dict__

{'value': 2, 'recent': [1]}

### 두번째 인스턴스에도 속성추가

In [13]:
a.number = 10
b.number = 3
print(b.number)

4


In [14]:
a.__dict__

{'value': 10, 'recent': [10]}

In [15]:
b.__dict__

{'value': 4, 'recent': [3]}

# 데이터 디스크립터  &  비 데이터 디스크립터


-  데이터 디스크립터 : `__set__`, `__delete__`중 하나라도 구현이 되어있으면 

-  비 데이터 디스크립터 : `__get__` 만 구현되어 있다면

In [16]:
from inspect import isfunction, isdatadescriptor

In [17]:
isfunction(lambda x : x)

True

In [18]:
class ND:
    def __get__(self, instance, owner):
        pass

In [19]:
class DD:
    def __get__(self, instance, owner):
        pass
    
    def __set__(self, instance, onwer):
        pass

In [20]:
nd = ND()
dd = DD()

In [21]:
isdatadescriptor(nd), isdatadescriptor(dd)

(False, True)

# 2. 디스크립터 속성 탐색

instance 속성 vs  class 속성

# 2.1 데이터 디스크립터로 속성을 정의한 경우

In [22]:
class DataDes:
    def __init__(self, value):
        self.value = value
        
    def __set__(self, instance, value):
        self.value = value
        
    def __get__(self, instance, owner):
        return self.value


In [23]:
class A:
    data = DataDes(3)

In [24]:
a = A()

In [25]:
a.data

3

In [26]:
a.__dict__

{}

### 객체 속성에 data 를 지정

In [27]:
a.__dict__['data'] = 9

### 객체에서 data를 참조하지만 클래스 속성인 디스크립터만 참조

In [28]:
a.data

3

In [29]:
a.__dict__

{'data': 9}

# 2.2 논데이터 디스크립터는 인스턴스의 dict보다 나중에 탐색

In [30]:
class nondata:
    def __init__(self):
        self.value = 3
        
    def __get__(self, instance, owner):
        return self.value
    

class A:
    nd = nondata()

In [31]:
a = A()

In [32]:
a.nd, a.__dict__

(3, {})

In [33]:
a.__dict__['nd'] = 100

In [34]:
a.nd

100

In [35]:
type(a).__dict__['nd'].__get__(a, type(a))

3

# 2.3 `__get__`이 없는 data descriptor는 본인을 리턴한다

In [36]:
class DD:
    def __init__(self, value):
        self.value = value
        
    def __set__(self, instance, value):
        self.value = value
        
    def __repr__(self):
        return '< HI >'

class A:
    data = DD(3)

In [37]:
a = A()

In [38]:
a.data

< HI >

In [39]:
a.data.value

3

In [40]:
a.data.__class__ is DD

True

# 2.4 클래스 dict의 값이 인스턴스 dict, 데이터디스크립터 보다 늦게 탐색

In [41]:
class A:
    def __init__(self, value):
        self.a = value
        
    a = 10

In [42]:
a = A(3)

In [43]:
A.__dict__, a.__dict__

(mappingproxy({'__module__': '__main__',
               '__init__': <function __main__.A.__init__(self, value)>,
               'a': 10,
               '__dict__': <attribute '__dict__' of 'A' objects>,
               '__weakref__': <attribute '__weakref__' of 'A' objects>,
               '__doc__': None}),
 {'a': 3})

In [44]:
a.a

3

# 2.5  상속인 경우 MRO 탐색

In [45]:
class DD:
    def __init__(self, value):
        self.value = value
        
    def __set__(self, instance, value):
        self.value = value
        
    def __get__(self, instance, owner):
        return self.value
    

class A:
    pass


class B:
    value = DD(100)
    
    
class C(A, B):
    pass

In [46]:
c = C()

In [47]:
type(c).__mro__

(__main__.C, __main__.A, __main__.B, object)

In [48]:
c.value

100

In [49]:
class A:
    value = 10
    

class B:
    value = DD(100)
    

class C(A, B):
    pass

In [50]:
c = C()
type(c).__mro__

(__main__.C, __main__.A, __main__.B, object)

In [51]:
c.value

10

# 3.  클래스 속성 탐색

## 메타클라스에 디스크립터 적용

In [52]:
class md:
    def __init__(self, value):
        self.value = value
        
    def __get__(self, instance, owner):
        print("md get")
        return self.value
    
    def __set__(self, instance, value):
        print('md set')
        self.value = value
        

class Meta(type):
    d = md(3)

In [53]:
class A(metaclass=Meta):
    pass

In [54]:
A.__dict__

mappingproxy({'__module__': '__main__',
              '__dict__': <attribute '__dict__' of 'A' objects>,
              '__weakref__': <attribute '__weakref__' of 'A' objects>,
              '__doc__': None})

In [55]:
A.d

md get


3

## 메타클래스에 `__get__`이 있는 데이터 디스크립터가 클래스의 값보다 우선한다.


In [56]:
class A(metaclass=Meta):
    d = 123

In [57]:
A.d

md get


3

## 메타클래스에 `__get__`이 없는 데이터디스크립터는 클래스의 속성보다 늦다.

In [58]:
class d1:
    def __init__(self, value):
        self.value = value
        
    def __set__(self, instance, value):
        self.value = value
        
    def __get__(self, instance, owner):
        return self.value
        
class m(type):
    value = d1(99)

In [59]:
class A(metaclass=m):
    value = 1

In [60]:
A.value

99

# set, delete 작동

In [61]:
class DD:
    def __init__(self, value):
        self.value = value
        
    def __set__(self, instance, value):
        raise AttributeError('NO!')
        
    def __get__(self, instance, owner):
        return self.value

class A:
    value = DD(99999)
    
    def __setattr__(self, name, value):
        print('setattr')
        super().__setattr__(name, value)

In [62]:
a = A()

In [63]:
a.value

99999

In [64]:
a.value = 123

setattr


AttributeError: NO!

###  클래스인 경우..

In [65]:
class DD:
    def __init__(self, value):
        self.value = value
        
    def __set__(self, instance, value):
        raise AttributeError('NO!')
        
    def __get__(self, instance, owner):
        return self.value

In [66]:
class M(type):
    value = DD(9123123)
    
    def __setattr__(self, name, value):
        print('class setattr')
        super().__setattr__(name, value)


In [67]:
class A(metaclass=M):
    pass

In [68]:
A.value

9123123

In [69]:
A.value = 123123

class setattr


AttributeError: NO!

### delete

In [70]:

class DD:
    def __init__(self, value):
        self.value = value
        
    def __delete__(self, instance):
        raise AttributeError('NO!')
        
    def __get__(self, instance, owner):
        return self.value

class A:
    value = DD(99999)
    
    def __delattr__(self, name):
        print('detattr')
        super().__delattr__(name,)

In [71]:
a = A()

In [72]:
a.value

99999

In [73]:
del a.value

detattr


AttributeError: NO!

# 4. 디스크립터를 사용할때, 정보를 저장하는 위치

In [74]:
class DD:
    # self = DD Class 인스턴스
    # instance = 디스크립터를 사용하는 클래스의 인스턴스 
    def __set__(self, instance, value):
        pass
    
    def __delete__(self, instance):
        pass
    
    def __get__(self, instance, owner):
        pass
    
        
class A:
    value = DD()

# 4.1 디스크립터에 저장

In [75]:
class descriptor:
    def __init__(self, value):
        self.value = value
        
    def __get__(self, instance, owner):
        return self.value
    
    def __set__(self, instance, value):
        self.value = value
        
class A:
    value = descriptor(10)

In [76]:
a = A()
b = A()

In [77]:
a.value

10

In [78]:
b.value

10

In [79]:
a.value = 10000

In [80]:
a.value

10000

In [81]:
b.value

10000

In [82]:
class descriptor:
    def __init__(self, value):
        self.storage = {}
        
    def __get__(self, instance, owner):
        return self.storage[instance]
    
    def __set__(self, instance, value):
        self.storage[instance] = value
        
    def __delete__(self, instance):
        del self.storage[instance]

In [83]:
class A:
    aaa = descriptor(123)

In [84]:
a = A()
b = A()
c = A()

In [85]:
a.aaa = 111

In [86]:
a.aaa

111

In [87]:
b.aaa = 222

In [88]:
c.aaa = 333

In [89]:
a.aaa, b.aaa, c.aaa

(111, 222, 333)

In [90]:
A.__dict__['aaa'].storage

{<__main__.A at 0x104729a90>: 111,
 <__main__.A at 0x1047e7150>: 222,
 <__main__.A at 0x1047e6550>: 333}

클래스 A가 생성한 모든 인스턴스에서 'aaa'이름으로 지정된 속성 값들을

모두 가져올 수 있다.

# 단점

In [91]:
del a

In [92]:
A.__dict__['aaa'].storage

{<__main__.A at 0x104729a90>: 111,
 <__main__.A at 0x1047e7150>: 222,
 <__main__.A at 0x1047e6550>: 333}

In [93]:
import weakref

In [94]:
wk_b = weakref.ref(b)

In [95]:
del b

In [96]:
wk_b

<weakref at 0x10481ef20; to 'A' at 0x1047e7150>

### 해결

In [97]:
from weakref import WeakKeyDictionary

In [98]:
dict_ = WeakKeyDictionary()

In [99]:
class A:
    pass

In [100]:
a = A()

In [101]:
dict_[a] = 123

In [102]:
dict_[a]

123

In [103]:
len(dict_)

1

In [104]:
del a

In [105]:
len(dict_)

0

In [106]:
from weakref import WeakKeyDictionary


class descriptor:
    def __init__(self, value):
        self.storage = WeakKeyDictionary()
        
    def __get__(self, instance, owner):
        return self.storage[instance]
    
    def __set__(self, instance, value):
        self.storage[instance] = value
        
    def __delete__(self, instance):
        del self.storage[instance]

In [107]:
from collections import UserDict

class A(UserDict):
    d = descriptor(10)

In [108]:
a = A()

In [109]:
a.d = 10

TypeError: unhashable type: 'A'

## id를 사용

In [110]:
d = {1, 2, 3}

In [111]:
d

{1, 2, 3}

In [112]:
hash(d)

TypeError: unhashable type: 'set'

In [113]:
a = {}

In [114]:
a[d]

TypeError: unhashable type: 'set'

In [115]:
a[id(d)] = 123

In [116]:
a

{4364937120: 123}

In [117]:
from weakref import finalize

In [118]:
class B:
    pass

b = B()

In [119]:
finalize(b, print, '제거', '합니다')

<finalize object at 0x10428e8e0; for 'B' at 0x1047714d0>

In [120]:
del b

제거 합니다


# 4.2 인스턴스 속성에 저장하는 디스크립터

## 디스크립터 클래스에서 값을 객체 속성에 할당

- 객체 내의 속성을 하나만 정의
- 여러 속성을 처리할 때는 이슈 발생

In [121]:
class dd:
    def __set__(self, instance, value):
        instance.data = value
        
    def __get__(self, instance, value):
        return instance.data
        
    def __delete__(self, instance):
        del instance.data

In [122]:
class A:
    d = dd()

### 2 객체를 생성해 속성 정의

In [123]:
a = A()
b = A()

In [124]:
a.d = 123
b.d = 10

In [125]:
a.d, b.d

(123, 10)

### 여러 개의 속성에 디스크립터를 만들 경우 부작용

- 하나의 객체 속성에만 값을 저장
- 여러 속성의 값이 하나의 객체 속성에 저장됨

In [126]:
class A:
    data1 = dd()
    data2 = dd()

In [127]:
a = A()

In [128]:
a.data1 = 123
a.data2 = 1999999

In [129]:
a.data1, a.data2

(1999999, 1999999)

In [130]:
a.__dict__

{'data': 1999999}

## 여러 속성을 정의할 경우 각 속성별로 이름을 배정

- __set_name__ 메소드를 정의해서 실제 속성이름을 가져와서 객체 속성의 이름으로 처리  


In [131]:
class dd:
    def __set_name__(self, owner, name):
        self.name = name
    
    def __set__(self, instance, value):
        instance.__dict__[self.name] = value
        
    def __get__(self, instance, value):
        return instance.__dict__[self.name]
        
    def __delete__(self, instance):
        del instance.__dict__[self.name]

In [132]:
class A:
    data1 = dd()
    data2 = dd()

In [133]:
a = A()

In [134]:
a.data1 = 123
a.data2 = 321

In [135]:
a.data1, a.data2

(123, 321)

In [136]:
a.__dict__

{'data1': 123, 'data2': 321}

# 5. 디스크립터에 추가

# 5.1 디스크립터 클래스 정의할 때 초기화에서 타입 정의

- 값을 갱신할 때 이 타입을 확인해서 처리함 

In [137]:
class Validated:
    def __set_name__(self, owner, name):
        self.name = name

    def __init__(self, type_):
        self.type = type_
        
    def __set__(self, instance, value):
        if isinstance(value, self.type):
            instance.__dict__[self.name] = value
        else:
            raise AttributeError('NO')

In [138]:
class A:
    num1 = Validated(int)
    num2 = Validated(float)

In [139]:
a = A()

In [140]:
a.num1 = 123
a.num2 = 1.0

In [141]:
a.num1, a.num2

(123, 1.0)

In [142]:
a.num1 = 1.0
a.num2 = 123

AttributeError: NO

# 5.2  특정 함수를 한번만 실행하도록 만들기

In [143]:
class lazy:
    def __init__(self, func):
        self.func = func
        
    def __get__(self, instance, owner):
        print("한번만 실행")
        value = self.func(instance)
        instance.__dict__[self.func.__name__] = value
        return value

In [144]:
class A:
    def __init__(self, value):
        self.value = value

    @lazy
    def data(self):
        return self.value * 10

In [145]:
a = A(111)

In [146]:
a.data

한번만 실행


1110

In [147]:
a.data

1110

In [148]:
a.data

1110

# 5.3  리드온리 

In [149]:
class descriptor:
    def __init__(self, value):
        self.value = value
    
    def __get__(self, instance, onwer):
        return self.value

    def __set__(self, instance, value):
        raise AttributeError('No')


In [150]:
class A:
    d = descriptor(10)

In [151]:
a = A()

In [152]:
a.d

10

In [153]:
a.__dict__

{}

In [154]:
a.d = 123

AttributeError: No

In [155]:
a.__dict__

{}

# 5.4 상수



In [156]:
class Constant:
    def __init__(self, value):
        self.value = value
        
    def __get__(self, instance, owner):
        return self.value
    
    def __set__(self, instance, value):
        raise AttributeError('no')
        
    def __delete__(self, instance):
        raise AttributeError('no')


In [157]:
class ConsDataMixin:
    PI = Constant(3.1415)
    e = Constant(2.71828)

In [158]:
class MathMeta(type, ConsDataMixin):
    pass

In [159]:
class Math(ConsDataMixin, metaclass=MathMeta):
    pass

In [160]:
Math.PI

3.1415

In [161]:
Math.PI = 123

AttributeError: no

In [162]:
math = Math()

In [163]:
math.PI

3.1415

In [164]:
math.PI = 123

AttributeError: no

`__getattribute__`, `__getattr__`, `__setattr__`